In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import functools

Consider the following problem:
\begin{align}
-\partial_x(\kappa(x) \partial_x u_x) & = f(x)\quad\text{in $[0,1]$}\\
u(0) = \alpha\\
u'(1) = \beta
\end{align}

In [ ]:
def f(x):
    #u = np.exp(-100 * (x - 0.25)**2)
    u = np.ones_like(x)
    return u

def kappa(x):
    u = 1 - 0.5*np.exp(-100 * (x - 0.75)**2)
    return u

alpha = 0.25
beta = 0.0

class Mesh:
    def __init__(self, nel):
        self.nel = nel
        x = np.linspace(0, 1, nel+1)
        self.x = x
        self.h = x[1] - x[0]
        self.n = len(x)

def gnorm(mesh, u):
    return np.sqrt(mesh.h) * np.linalg.norm(u)

In [ ]:
def assemble(mesh, f, kappa):
    n = mesh.n
    nel = mesh.nel
    h = mesh.h
    x = mesh.x
    
    A = np.zeros((n, n))
    rhs = np.zeros(n)
    for e in range(0, nel):
        AA = np.array([[ 1/h, -1/h],
                       [-1/h,  1/h]])
        #ff = np.array([h/2, h/2])       
        t = 0.5
        ff0 = f(h * t + x[e])*(1-t)*h
        ff1 = f(h * t + x[e])*(t)*h
        ff = np.array([ff0, ff1])
        
        if e == 0:
            AA[:,0] = 0
            AA[0,:] = 0
            ff[0] = 0
        
        A[e:e+2, e:e+2] += AA
        rhs[e:e+2] += ff
    A = A[1:, 1:]
    rhs = rhs[1:]
    return A, rhs

Questions that you should consider:
* What if $\kappa(x) \ne 1$; how do we do integration?
* What if $f(x)$ is complicated; how do we do integration?
* How do we impose $u(0) = a$ on the space/problem?
* What tests can you include to verify your implementation?
  
Challenge questions:
* How do we use a sparse matrix instead of a numpy array?
* What if $\beta \ne 0$?  How do we handle this in the the problem?
* What if we use a quadratic basis vs a linear basis?

In [ ]:
def mytest(test):
    @functools.wraps(test)
    def wrapper():
        test()
        print(f'{test.__name__}: pass')
    return wrapper

@mytest
def test1():
    mesh = Mesh(3)
    f = lambda x: 1
    kappa = lambda x: 1
    A, rhs = assemble(mesh, f, kappa)
    Aref = np.array([[2, -1, 0],
                     [-1, 2, -1],
                     [0, -1, 1]])
    rhsref = np.array([2, 2, 1])
    np.testing.assert_array_almost_equal(mesh.h*A, Aref)
    np.testing.assert_array_almost_equal(2*rhs/mesh.h, rhsref)

@mytest
def test2():
    def f(x):
        return (np.pi**2/4)*np.sin(np.pi*x/2)
    kappa = lambda x: 1
    def uexact(x):
        return np.sin(np.pi*x/2)

    e0 = 1
    for nel in [8, 16, 32, 64, 128]:
        mesh = Mesh(nel)
        A, rhs = assemble(mesh, f, kappa)
        u = np.linalg.solve(A, rhs)
        e = gnorm(mesh, u - uexact(mesh.x[1:]))
        ratio = e0/e
        e0 = e
    assert ratio > 3.5

test1()
test2()

In [ ]:
mesh = Mesh(200)
def f(x):
    return (np.pi**2/4)*np.sin(np.pi*x/2)
A, rhs = assemble(mesh, f, kappa)
u = np.linalg.solve(A, rhs)
plt.plot(mesh.x[1:], u)
plt.plot(mesh.x[1:], np.sin(np.pi*mesh.x[1:]/2))